First, start from treating pbc using GROMACS.

In [ ]:
import sys, os
  
for i in range(20):
    os.chdir('%d'%i)
    os.system('echo "2\n" | gmx trjconv -f traj_comp.xtc -n index -o traj_protein.xtc')  # get protein only traj
    os.system('echo "2\n" | gmx trjconv -s protein.tpr -f traj_protein.xtc -o pbc_mol.xtc -pbc mol -n index')  # put separate parts back as a whole
    os.system('echo "11\n 2\n" | gmx trjconv -s protein.tpr -f pbc_mol.xtc -n index.ndx -o center.xtc -center')  # center the system
    os.chdir('../')


However, this doesn't work as expected...

Then, try following:

In [ ]:
for i in range(20):
    os.chdir('%d'%i)
    os.system('echo "2\n 2\n" | gmx trjconv -s protein.tpr -f traj_protein.xtc -o pbc_cluster.xtc -pbc cluster -n index')
    os.system('echo "11\n 2\n" | gmx trjconv -s protein.tpr -f pbc_cluster.xtc -n index.ndx -o cluster_center.xtc -center')
    os.chdir('../')


By loading trajectories in VMD, it seems this protocol works!

Now, align these traj based upon the reference structure

In [ ]:
import sys, os
import mdtraj as md


ref = md.load('new_test.pdb')
for i in range(20):
    os.chdir('%d'%i)
    t=md.load('cluster_center.xtc',top='protein.gro')
    t_new = t.superpose(ref)
    t_new.save_xtc('aligned_cluster.xtc')
    os.chdir('../')


Now, lets treat pbc again to move from original box to a new box (-1.5,1.5)

In [ ]:
import sys, os
import mdtraj as md
from matplotlib import pyplot as plt
from mdtraj.formats import XTCTrajectoryFile
import numpy as np

for run in range(20):
    print run
    os.chdir('%d'%run)
    raw = md.load('aligned_cluster.xtc',top='../new_test.pdb')
    new_xyz = np.zeros((raw.xyz.shape))
    n_frame = raw.xyz.shape[0]
    n_atom = raw.xyz.shape[1]
    n_coor = raw.xyz.shape[2]

    for i in range(n_frame):
        for j in range(n_atom):
            for k in range(n_coor):
                if raw.xyz[i,j,k] > 3.00859/2.:
                    while raw.xyz[i,j,k] > 3.00859/2.:
                        raw.xyz[i,j,k] = raw.xyz[i,j,k] - 3.00859
                    new_xyz[i,j,k] = raw.xyz[i,j,k]
                elif raw.xyz[i,j,k] < -3.00859/2.:
                    while raw.xyz[i,j,k] < -3.00859/2.:
                        raw.xyz[i,j,k] = raw.xyz[i,j,k] + 3.00859
                    new_xyz[i,j,k] = raw.xyz[i,j,k]
                else:
                    new_xyz[i,j,k] = raw.xyz[i,j,k]

    with XTCTrajectoryFile('pbc_treat_traj_cluster.xtc','w') as f:
        f.write(new_xyz)
    np.save('pbc_treat_traj_cluster.npy',new_xyz)
    os.chdir('../')


Let's check if there are any atoms outside the new box.

In [ ]:
import sys, os
import numpy as np
from matplotlib import pyplot as plt

data = []

for run in range(20):
    os.chdir('%d'%run)
    t = np.load('pbc_treat_traj_cluster.npy')
    for i in range(12):
        for j in range(3):
            data.append(max(t[:,i][:,j]))   # find max and min values of x,y,z of all atoms in this trajectory
            data.append(min(t[:,i][:,j]))
    os.chdir('../')
np.save('max_min_coord_cluster.npy',data)
plt.figure()
x = range(len(data))
plt.plot([0,max(x)],[3.00859/2.,3.00859/2.],color='red',linestyle='-')
plt.plot([0,max(x)],[-3.00859/2.,-3.00859/2.],color='red',linestyle='-')
plt.plot(x,data,'o',color='blue')
plt.xlabel('data')
plt.ylabel('coordinate')
plt.savefig('figure_checkpbc_cluster.png')


Let's also check if r=np.sqrt(x^2.+y^2.+z^2) can be used as a good quantity as distance between the liagnd and the center of mass of the pocket.

In [ ]:
import sys, os
import mdtraj as md
import numpy as np


dis_center = [[] for i in range(20)]
dis=[[] for i in range(20)]

for j in range(20):
    t = md.load('%d/pbc_treat_traj_cluster.xtc'%j,top='new_test.pdb')
    ind=t.topology.select('(index 0 to 7) or (index 9 to 10)')
    ind2 = t.topology.select('index 11')
    t1=t.atom_slice(ind)
    t2=t.atom_slice(ind2)
    com1=md.compute_center_of_mass(t1)
    com2=md.compute_center_of_mass(t2)
    for i in range(len(com1)):
        dis_center[j].append(np.sqrt((com1[i][0]-com2[i][0])**2.0 + (com1[i][1]-com2[i][1])**2.0 + (com1[i][2]-com2[i][2])**2.0))
        dis[j].append(np.sqrt(com2[i][0]**2.0+com2[i][1]**2.0+com2[i][2]**2.0))
np.save('ligand_dis_com_cluster.npy',dis_center)
np.save('ligand_dis_abs_cluster.npy',dis)


Let's plot the difference of these two types of distances and plot the histogram of the differences.

In [ ]:
import sys, os
import numpy as np
from matplotlib import pyplot as plt

abs_dis = np.load('ligand_dis_abs_cluster.npy')
com_dis = np.load('ligand_dis_com_cluster.npy')

diff = np.zeros((abs_dis.shape))

for i in range(len(abs_dis)):
    for j in range(len(abs_dis[i])):
        diff[i,j] = abs_dis[i][j] - com_dis[i][j]
np.save('dis_diff_cluster.npy',diff)
plt.figure(figsize=(20,25))
for i in range(len(diff)):
    print i
    plt.subplot(4,5,i+1)
    counts,bins = np.histogram(diff[i],bins=np.arange(min(diff[i]),max(diff[i]),0.01))
    bincenters = (bins[0:-1]+bins[1:])/2.0
    plt.plot(bincenters,counts)
    plt.xlabel('diff (nm)',fontsize=5)
    plt.yticks([])
    plt.xticks(fontsize=5)
plt.tight_layout()
plt.savefig('diff_cluster.pdf')


Let's compute distance of the ligand between frames.

In [ ]:
import sys, os
import numpy as np

dis = [[] for i in range(20)]
for i in range(20):
    os.chdir('%d'%i)
    t=np.load('pbc_treat_traj_cluster.npy')
    for j in range(len(t)-1):
        dis[i].append(np.sqrt((t[:,11][j+1][0]-t[:,11][j][0])**2.0 + (t[:,11][j+1][1]-t[:,11][j][1])**2.0 +(t[:,11][j+1][2]-t[:,11][j][2])**2.0))
    os.chdir('../')
np.save('fram_distance_cluster.npy',dis)


Let's take a look at the distribution of these distances between frames.

In [ ]:
import sys, os
import numpy as np
from matplotlib import pyplot as plt

dis = np.load('frame_distance_cluster.npy')
plt.figure(figsize=(20,25))
for i in range(len(dis)):
    print i
    plt.subplot(4,5,i+1)
    counts,bins = np.histogram(dis[i],bins=np.arange(min(dis[i]),max(dis[i]),0.01))
    bincenters = (bins[0:-1]+bins[1:])/2.0
    plt.plot(bincenters,counts)
    plt.xlabel('distance (nm)',fontsize=5)
    plt.yticks([])
    plt.xticks(fontsize=5)
plt.tight_layout()
plt.savefig('frame_dis_cluster.pdf')

zoom = []

for i in range(len(dis)):
    for j in range(len(dis[i])):
        if dis[i][j] > 2.0:
            zoom.append(dis[i][j])
plt.figure()
counts,bins = np.histogram(zoom,bins=np.arange(2.0,3.1,0.001))
bincenters = (bins[0:-1]+bins[1:])/2.0
plt.plot(bincenters,counts)
plt.xlabel('distance (nm)')
plt.savefig('frame_dis_zoom_cluster.pdf')


From the previous plot, let's find the frames that have large distances (>2.4 nm) which is considered as a crossing box event.

In [ ]:
import sys, os
import numpy as np

dis = np.load('frame_distance_cluster.npy')
cross = [[] for i in range(20)]

for i in range(len(dis)):
    for j in range(len(dis[i])):
        if dis[i][j] >= 2.4:
            cross[i].append(j+1)
np.save('cross_ind_cluster.npy',cross)


Now, let's separate trajectories into pieces without crossing box event.

In [ ]:
import sys, os
import numpy as np
from itertools import groupby
from operator import itemgetter

cross = np.load('cross_ind_cluster.npy')
cross_new = [[] for i in range(len(cross))]

for i in range(len(cross)):
    for k, g in groupby(enumerate(cross[i]), lambda (i, x): i-x):
        cross_new[i].append(list(map(itemgetter(1),g)))


for r in range(20):
    print r
    traj = np.load('%d/pbc_treat_traj_cluster.npy'%r)
    new_traj = []
    for i in range(len(cross_new[r])):
        if i == 0:
            new_traj.append(traj[:,11][0:cross_new[r][i][0]])
        elif i == len(cross_new[r])-1:
            new_traj.append(traj[:,11][cross_new[r][i][-1]:])
        else:
            new_traj.append(traj[:,11][(cross_new[r][i-1][-1]+1):cross_new[r][i][0]])
    np.save('%d/sep_traj_cluster.npy'%r,new_traj)



Let's compute r, cos_theta, sin_phi, cos_phi

In [ ]:
import sys, os
import numpy as np

for i in range(20):
    print i
    xyz = np.load('%d/sep_traj_cluster.npy'%i)
    new_xyz = []
    for j in range(len(xyz)):
        if len(xyz[j]) >= 20.:
            new_xyz.append(xyz[j])
    n_traj = len(new_xyz)
    spherical = []
    for k in range(n_traj):
        sphe_sep = np.zeros((len(new_xyz[k]),4))
        for l in range(len(new_xyz[k])):
            if new_xyz[k][l][0] == 0. and new_xyz[k][l][1] == 0. and new_xyz[k][l][2] == 0.:
                r = 0.
                cos_theta = 0.
                cos_phi = 0.
                sin_phi = 0.
            else:
                r = np.sqrt(new_xyz[k][l][0]**2.+new_xyz[k][l][1]**2.+new_xyz[k][l][2]**2.)
                cos_theta = new_xyz[k][l][2]/r
                if new_xyz[k][l][0] == 0. and new_xyz[k][l][1] == 0.:
                    cos_phi = 0.
                    sin_phi = 0.
                else:
                    cos_phi = new_xyz[k][l][0]/np.sqrt(new_xyz[k][l][0]**2.+new_xyz[k][l][1]**2.)
                    sin_phi = new_xyz[k][l][1]/np.sqrt(new_xyz[k][l][0]**2.+new_xyz[k][l][1]**2.)
            sphe_sep[l][0] = r
            sphe_sep[l][1] = cos_theta
            sphe_sep[l][2] = cos_phi
            sphe_sep[l][3] = sin_phi
        spherical.append(sphe_sep)
    np.save('spher/cluster/spher_%d.npy'%i,spherical)


MSM construction...

In [ ]:
from msmbuilder.featurizer import DihedralFeaturizer
from msmbuilder.dataset import dataset
import numpy as np
from matplotlib import pyplot as plt
import mdtraj as md
import os,sys, glob
import msmbuilder.utils
import pickle
from msmbuilder.utils import load
from msmbuilder.cluster import KMeans
from msmbuilder.cluster import KCenters
from msmbuilder.cluster import KMedoids
from msmbuilder.cluster import MiniBatchKMeans
from msmbuilder.msm import implied_timescales
from msmbuilder.msm import ContinuousTimeMSM, MarkovStateModel
from itertools import combinations
from msmbuilder.featurizer import AtomPairsFeaturizer
from msmbuilder.decomposition import tICA
from sklearn.pipeline import Pipeline
from msmbuilder.example_datasets import fetch_met_enkephalin
from matplotlib import pyplot as plt
from sklearn.externals import joblib

distances=[]
for i in range(20):
        print i
        b=np.load('../../spher/cluster/spher_%d.npy'%i)
        for j in range(len(b)):
            distances.append(b[j])
tica_model = tICA(lag_time=5,n_components=4)
tica_fit = tica_model.fit(distances)
output=open('tica_model.npy','wb')
pickle.dump(tica_model,output)
output.close()
tica_transformed = tica_fit.transform(distances)
output=open('tica_eigenvectors.npy','wb')
pickle.dump(tica_model.eigenvectors_,output)
output.close()
np.save('tica.npy',tica_transformed)


Clustering...

In [ ]:
tica_transformed=np.load('tica.npy')
assignments=KCenters(n_clusters=20).fit(tica_transformed)
centers=assignments.cluster_centers_
np.save('cluster_centers.npy',centers)
ass=assignments.transform(tica_transformed)
np.save('Assignments.npy',ass)
lagtimes = np.array([1,5,10,15,30,100,150,200,300,600,1000,1200,1500,2000,2500,5000,8000,10000,15000,20000, 30000,40000,50000,60000,70000,80000,90000,100000])
msmts = []
for lagtime in lagtimes:
        print "\tLagtime: %d"%lagtime
        msm = MarkovStateModel(lag_time=lagtime).fit(ass)
        msmts.append(msm.timescales_)
np.save('test_msmts.npy', msmts)


ns_per_frame = 0.001
import matplotlib.pyplot as plt
plt.figure()
for j in range(4):
        print j
        plt.plot(ns_per_frame*lagtimes,[ns_per_frame*msmts[i][j] for i in range(len(lagtimes))])

plt.xlabel('lagtime (ns)')
plt.ylabel('implied timescale (ns)')
plt.yscale('log')
plt.savefig('lagtime_transform.pdf')
plt.show()

